In [2]:
import pandas as pd 
import numpy as np
from nilearn import input_data
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
import glob
from tqdm import tqdm
from nilearn.connectome import ConnectivityMeasure
import os

In [3]:
dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm', symmetric_split= True)
atlas_filename = dataset.maps
labels = dataset.labels[1:]
masker_cor = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                           memory='nilearn_cache', verbose=0)

# create masker for HO subcortical
dataset = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm', symmetric_split= True)
atlas_filename = dataset.maps
labels_s = dataset.labels[1:]

masker_sub = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                           memory='nilearn_cache', verbose=0)

In [9]:
df = pd.read_csv('csvfiles/abide.csv')


atlas_name = 'HO_sub'
for i in tqdm(range(len(df))):
    fmri_filename = df['RSFMRI_file'].iloc[i]
    sub_path = df['tc_file'].iloc[i].replace('ATLAS', atlas_name).replace('/timecourse.csv', '')
    if not os.path.isdir(sub_path):
        os.makedirs(sub_path)

        # extract timeseries and concatenate them
        time_series_c = masker_cor.fit_transform(fmri_filename)
        time_series_s = masker_sub.fit_transform(fmri_filename)
        tc = np.concatenate((time_series_c,time_series_s), 1)



        correlation_measure = ConnectivityMeasure(kind='correlation')
        cc = correlation_measure.fit_transform([tc])[0]

        np.save(sub_path+'/tc.npy',tc)
        np.save(sub_path+'/cc.npy',cc)

 16%|████████████████████████▏                                                                                                                           | 197/1207 [21:11<1:45:15,  6.25s/it]/data/agelgazzar/anaconda3/envs/ssm/lib/python3.9/site-packages/nilearn/maskers/nifti_labels_masker.py:492: UserWarning: Persisting input arguments took 0.55s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  region_signals, labels_ = self._cache(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1207/1207 [2:09:21<00:00,  6.43s/it]
